In [ ]:
import folium
import numpy as np
from folium.plugins import HeatMap
from scipy.stats import gaussian_kde
import polars as pl
import geopandas as gpd

import fryer.data as data
import fryer.map as map

from shapely.geometry import Point

In [ ]:
collision = data.uk_gov_dept_for_transport_road_accidents.read_collision()
gdf = data.ons_local_authority_district_boundaries.read()

In [ ]:
local_authority_ons_code = "E09000003"

In [ ]:
gdf.sample(1)

In [ ]:
local_authority_df = (
    collision.filter(
        pl.col("local_authority_ons_district") == local_authority_ons_code,
        pl.col("latitude").is_not_null(),
        pl.col("longitude").is_not_null(),
    )
).to_pandas()

In [ ]:
from folium import FeatureGroup, CircleMarker

# 1. Create a Folium map centered around the UK
uk_map = map.create_map()

# ----------------------------
# 2. Add Local Authority Boundary with Transparent Fill and Bold Black Border
# ----------------------------
boundary_df = gdf[gdf["LAD24CD"] == local_authority_ons_code]
sim_geo = gpd.GeoSeries(boundary_df["geometry"]).simplify(tolerance=0.001)
geo_j = sim_geo.to_json()

geo_j = folium.GeoJson(
    data=geo_j,
    style_function=lambda x: {
        "fillColor": "transparent",  # Transparent fill
        "fillOpacity": 0.0,  # Ensure fill is fully transparent
        "color": "black",  # Bold black border
        "weight": 3,  # Border thickness
        "opacity": 1.0,  # Border opacity
    },
)
folium.Popup(boundary_df["LAD24NM"]).add_to(geo_j)
geo_j.add_to(uk_map)

# ----------------------------
# 3. Extract coordinates for KDE
# ----------------------------
latitudes = local_authority_df["latitude"]
longitudes = local_authority_df["longitude"]

# Perform Kernel Density Estimation (KDE)
kde = gaussian_kde([longitudes, latitudes], bw_method="silverman")

# Generate a grid of points for KDE
n_points = 200  # Adjust for more resolution
lon_min, lon_max = min(longitudes), max(longitudes)
lat_min, lat_max = min(latitudes), max(latitudes)

grid_lon = np.linspace(lon_min, lon_max, n_points)
grid_lat = np.linspace(lat_min, lat_max, n_points)
lon_grid, lat_grid = np.meshgrid(grid_lon, grid_lat)
grid_points = np.vstack([lon_grid.flatten(), lat_grid.flatten()])

# Calculate KDE values
density_values = kde(grid_points)
density_values = density_values.reshape(lon_grid.shape)
density_values = density_values / np.max(density_values)  # Normalize

# ----------------------------
# 4. Clip KDE Grid Points to Local Authority Polygon
# ----------------------------
boundary_polygon = boundary_df.geometry.iloc[0]  # Extract the polygon geometry

# Create a list of valid heatmap points
heat_data = []
for i in range(len(density_values.flatten())):
    lat = lat_grid.flatten()[i]
    lon = lon_grid.flatten()[i]
    point = Point(lon, lat)
    if boundary_polygon.contains(point):
        intensity = density_values.flatten()[i]
        heat_data.append([lat, lon, intensity])

# Add Clipped HeatMap to the Folium Map
HeatMap(heat_data, min_opacity=0.2, radius=25, blur=20, name="Accident Heatmap").add_to(
    uk_map
)

# ----------------------------
# 5. Add Individual Accident Points as a Toggleable Layer
# ----------------------------
accident_points_layer = FeatureGroup(name="Accident Points")

for idx, row in local_authority_df.iterrows():
    CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=1,  # Adjust size for visibility
        color="red",
        fill=True,
        fill_opacity=0.8,
        popup=folium.Popup(f"Accident ID: {row['accident_index']}", max_width=200),
    ).add_to(accident_points_layer)

# Add the accident points layer to the map
accident_points_layer.add_to(uk_map)

# ----------------------------
# 6. Add Layer Control
# ----------------------------
folium.LayerControl().add_to(uk_map)

In [ ]:
uk_map